<a href="https://colab.research.google.com/github/MathotaGamma/AI/blob/main/CivitAI5-2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# --- モード選択 ---
print("=== モード選択 ===")
print("1: Hugging Faceで画像生成")
print("2: CivitAI（モデルDL / safetensors変換）")
print("3: トークン数を数える")
print("4: 出力フォルダを削除")
mode = input("番号を入力してください: ").strip()

# --- 必要なパラメータを先に全て入力 ---
hf_params = {}
civitai_params = {}

model_list_H = ['stablediffusionapi/eleet-model',
              'stablediffusionapi/brav6',
              'Vsukiyaki/ShiratakiMix',
              'stabilityai/stable-diffusion-xl-base-1.0'
              ]

model_list_C = ["https://civitai.com/api/download/models/28100?type=Model&format=SafeTensor&size=full&fp=fp16",
                "https://civitai.com/api/download/models/11812?type=Model&format=SafeTensor&size=full&fp=fp16"
              ]



if mode == "1":
  hf_params["model_id"] = input("HuggingFaceのモデルIDを入力（例: runwayml/stable-diffusion-v1-5）: ").strip()
  hf_params["submode"] = input("生成モードを選択（1: text2img, 2: img2img）: ").strip()
  hf_params["prompt"] = input("プロンプトを入力: ").strip()
  hf_params["negative_prompt"] = input("ネガティブプロンプトを入力 : ").strip()
  hf_params["num"] = int(input("生成する画像の枚数 : "))
  hf_params["steps"] = int(input("ステップ数（例: 70）: "))
  hf_params["guidance"] = float(input("guidance scale（例: 7.5）: "))
  if hf_params["submode"] == "2":
    hf_params["strength"] = float(input("strength (例: 0.75) : "))
    hf_params["image_path"] = input("初期画像のパスを入力してください（img2img用）: ")

elif mode == "2":
  print("--- CivitAI モデルの状況は？ ---")
  print("1: まだ何もしていない（DLも変換もしていない）")
  print("2: safetensorsファイルのみアップロード済み（変換が必要）")
  print("3: 変換済みでそのまま使える")
  civitai_params["status"] = input("番号を入力してください: ").strip()
  #civitai_params["model_name"] = input("保存先モデル名（例: models）: ").strip()
  if civitai_params["status"] == "1":
    civitai_params["url"] = input("CivitAIのモデル/LoRA ダウンロードURL: ").strip()
    civitai_params["need_yaml"] = input("v1-inference.yamlが必要ですか？ (y/n): ").strip().lower() == "y"
    civitai_params["safetensorsName"] = input("ファイル名(downloaded_model.safetensors) : ")
    if civitai_params["safetensorsName"] == "":
      civitai_params["safetensorsName"] = "downloaded_model.safetensors"
  elif civitai_params["status"] == "2":
    civitai_params["safefile"] = input("アップロード済みの .safetensors ファイル名(downloaded_model.safetensors): ").strip()
    if civitai_params["safefile"] == "":
      civitai_params["safefile"] = "downloaded_model.safetensors"
    civitai_params["need_yaml"] = input("v1-inference.yamlが必要ですか？ (y/n): ").strip().lower() == "y"
  civitai_params["prompt"] = input("プロンプトを入力してください: ")
  civitai_params["negative_prompt"] = input("ネガティブプロンプトを入力 : ").strip()
  civitai_params["num"] = int(input("生成する画像の枚数 : "))
  civitai_params["steps"] = int(input("ステップ数（例: 70）: "))
  civitai_params["lora_path"] = input("LoRAを使用する場合はpathを(使わない場合は無入力)")
  if civitai_params["lora_path"] == "":
    civitai_params["lora_path"] = None

elif mode == "3":
  token_prompt = input("プロンプトを入力してください（トークン数確認）: ").strip()

elif mode == "4":
  confirm_delete = input("本当にoutputフォルダを削除しますか？ (y/n): ").strip().lower()

# --- 必要なinstallはここで実行（全てのinput終了後） ---
print("インストールを開始します...")
!pip install diffusers transformers accelerate safetensors

# --- 各モード実行 ---
if mode == "1":
  from diffusers import StableDiffusionPipeline, StableDiffusionImg2ImgPipeline
  from PIL import Image
  import torch
  import os
  torch_dtype = torch.float16
  if hf_params["submode"] == "1":
    pipe = StableDiffusionPipeline.from_pretrained(
      hf_params["model_id"],
      torch_dtype=torch.float16,
      safety_checker=None  # Safe checkを無効に
    )
    pipe = pipe.to("cuda")  # GPUが必要です
  else:
    pipe = StableDiffusionImg2ImgPipeline.from_pretrained(
        hf_params["model_id"],
        torch_dtype=torch.float16,
        safety_checker=None
    )
    pipe = pipe.to("cuda")
  for k in range(hf_params["num"]):
    if hf_params["submode"] == "1":

      image = pipe(hf_params["prompt"],
                   negative_prompt=hf_params["negative_prompt"],
                   num_inference_steps=hf_params["steps"],
                   guidance_scale=hf_params["guidance"],
                   added_cond_kwargs={}).images[0]
    else:
      init_image = Image.open(hf_params["image_path"]).convert("RGB").resize((512, 512))
      image = pipe(prompt=hf_params["prompt"],
                   negative_prompt=hf_params["negative_prompt"],
                   num_inference_steps=hf_params["steps"],
                   image=init_image,
                   strength=hf_params["strength"],
                   guidance_scale=hf_params["guidance"]).images[0]

    os.makedirs("output", exist_ok=True)
    filename = f"output/huggingface_result"+str(k)+".png"
    image.save(filename)
  print("画像を保存しました")

elif mode == "2":
  import os
  import torch
  import re
  from diffusers import StableDiffusionPipeline

  base_dir = "/content/models"
  os.makedirs(base_dir, exist_ok=True)

  if civitai_params["status"] == "1":
    regex = r'([0-9]{5})'
    if bool(re.match(regex, civitai_params['url'])):
      civitai_params['url'] = 'https://civitai.com/api/download/models/'+civitai_params['url']
    elif civitai_params['url'] == '1':
      civitai_params['url'] = "https://civitai.com/api/download/models/28100?type=Model&format=SafeTensor&size=full&fp=fp16"
    elif civitai_params['url'] == '2':
      civitai_params['url'] = "https://civitai.com/api/download/models/11812?type=Model&format=SafeTensor&size=full&fp=fp16"


    download_path = os.path.join(base_dir, civitai_params["safetensorsName"])

    model_url = civitai_params["url"]
    model_name = civitai_params["safetensorsName"]

    print(f"Downloading Checkpoint model to {download_path}...")
    !wget -O {download_path} "{model_url}"
    print(f"Download of {model_name} complete!")

    # Checkpointモデルのパスを記録
    checkpoint_path = download_path
    lora_path = civitai_params["lora_path"] # LoRAはダウンロードしないのでNone


    if civitai_params["need_yaml"]:
      !wget -O /content/models/v1-inference.yaml \
https://raw.githubusercontent.com/CompVis/stable-diffusion/main/configs/stable-diffusion/v1-inference.yaml
    !mkdir /content/converted
    !wget -O convert_original_stable_diffusion_to_diffusers.py \
    https://raw.githubusercontent.com/huggingface/diffusers/main/scripts/convert_original_stable_diffusion_to_diffusers.py
    os.makedirs(base_dir, exist_ok=True)
    !python convert_original_stable_diffusion_to_diffusers.py \
      --checkpoint_path ./models/downloaded_model.safetensors \
      --original_config_file ./models/v1-inference.yaml \
      --dump_path ./converted/diffusers_model \
      --from_safetensors

  elif civitai_params["status"] == "2":
    if civitai_params["need_yaml"]:
      !wget -O /content/models/v1-inference.yaml \
https://raw.githubusercontent.com/CompVis/stable-diffusion/main/configs/stable-diffusion/v1-inference.yaml
      #!wget -q https://huggingface.co/runwayml/stable-diffusion-v1-5/resolve/main/v1-inference.yaml
    !mkdir /content/converted
    !wget -O convert_original_stable_diffusion_to_diffusers.py \
    https://raw.githubusercontent.com/huggingface/diffusers/main/scripts/convert_original_stable_diffusion_to_diffusers.py
    os.makedirs(base_dir, exist_ok=True)
    !python convert_original_stable_diffusion_to_diffusers.py \
      --checkpoint_path ./models/downloaded_model.safetensors \
      --original_config_file ./models/v1-inference.yaml \
      --dump_path ./converted/diffusers_model \
      --from_safetensors

  #pipe = StableDiffusionPipeline.from_pretrained(base_dir, torch_dtype=torch.float16).to("cuda")
  pipe = StableDiffusionPipeline.from_pretrained(
      "/content/converted/diffusers_model",
      torch_dtype=torch.float16,
      use_safetensors=False,
      safety_checker=None
    ).to("cuda")
  # パイプラインを作成した直後に以下を追加します
  if lora_path:
    pipe.load_lora_weights(lora_path, use_safetensors=True)

  #pipe.enable_xformers_memory_efficient_attention()
  for k in range(civitai_params["num"]):
    image = pipe(prompt=civitai_params["prompt"], negative_prompt=civitai_params["negative_prompt"], num_inference_steps=civitai_params["steps"]).images[0]
    os.makedirs("output/"+civitai_params["prompt"][0:10], exist_ok=True)
    image.save("output/"+civitai_params["prompt"][0:10]+"/"+str(k)+".png")
  print("画像を保存しました: output/"+civitai_params["prompt"][0:10]+".png")

elif mode == "3":
  from transformers import CLIPTokenizer
  tokenizer = CLIPTokenizer.from_pretrained("openai/clip-vit-large-patch14")
  tokens = tokenizer(token_prompt)["input_ids"]
  print(f"トークン数: {len(tokens)}")

elif mode == "4":
  import shutil
  if confirm_delete == "y" and os.path.exists("output"):
    shutil.rmtree("output")
    print("outputフォルダを削除しました")
  else:
    print("キャンセルされました")


=== モード選択 ===
1: Hugging Faceで画像生成
2: CivitAI（モデルDL / safetensors変換）
3: トークン数を数える
4: 出力フォルダを削除
番号を入力してください: 2
--- CivitAI モデルの状況は？ ---
1: まだ何もしていない（DLも変換もしていない）
2: safetensorsファイルのみアップロード済み（変換が必要）
3: 変換済みでそのまま使える
番号を入力してください: 3
プロンプトを入力してください: anime, sky blue and white hair, cute woman, active, in snowy, sunny, wearing coat, masterpiece, facing front, low ponytail
ネガティブプロンプトを入力 : real, low quality
生成する画像の枚数 : 15
ステップ数（例: 30）: 100
インストールを開始します...


Loading pipeline components...:   0%|          | 0/6 [00:00<?, ?it/s]

You have disabled the safety checker for <class 'diffusers.pipelines.stable_diffusion.pipeline_stable_diffusion.StableDiffusionPipeline'> by passing `safety_checker=None`. Ensure that you abide to the conditions of the Stable Diffusion license and do not expose unfiltered results in services or applications open to the public. Both the diffusers team and Hugging Face strongly recommend to keep the safety filter enabled in all public facing circumstances, disabling it only for use-cases that involve analyzing network behavior or auditing its results. For more information, please have a look at https://github.com/huggingface/diffusers/pull/254 .


  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

...

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
from diffusers import StableDiffusionPipeline
import torch
import tempfile


# モデル読み込み
pipe = StableDiffusionPipeline.from_single_file(
    '/content/models/downloaded_model.safetensors',
    torch_dtype=torch.float16,
    use_safetensors=True
    #variant="fp16"  # 明示しておくと安全
)
pipe.to("cuda")

# プロンプトで画像生成
image = pipe("masterpiece, best quality, 1girl").images[0]
image.save("output.png")


Fetching 17 files:   0%|          | 0/17 [00:00<?, ?it/s]

Loading pipeline components...:   0%|          | 0/5 [00:00<?, ?it/s]

You have disabled the safety checker for <class 'diffusers.pipelines.stable_diffusion.pipeline_stable_diffusion.StableDiffusionPipeline'> by passing `safety_checker=None`. Ensure that you abide to the conditions of the Stable Diffusion license and do not expose unfiltered results in services or applications open to the public. Both the diffusers team and Hugging Face strongly recommend to keep the safety filter enabled in all public facing circumstances, disabling it only for use-cases that involve analyzing network behavior or auditing its results. For more information, please have a look at https://github.com/huggingface/diffusers/pull/254 .


  0%|          | 0/50 [00:00<?, ?it/s]

TypeError: argument of type 'NoneType' is not iterable

In [ ]:
from diffusers import StableDiffusionPipeline
import torch
import tempfile
from transformers import CLIPTextModel, CLIPTokenizer

# モデル読み込み
pipe = StableDiffusionPipeline.from_single_file(
    '/content/models/downloaded_model.safetensors',
    torch_dtype=torch.float16,
    use_safetensors=True
    #variant="fp16"  # 明示しておくと安全
)
pipe.to("cuda")

# プロンプト
prompt = "masterpiece, best quality, 1girl"

# CLIPモデルとトークナイザーをロードしてテキスト埋め込みを生成 (SDXLの場合に必要)
# モデルによっては異なるCLIPモデルが必要になる場合があります。
# ロードしているsafetensorsファイルがどのモデルに基づいているか確認してください。
# ここでは例としてSDXLで使用されることが多いCLIPモデルをロードします。
try:
    tokenizer = CLIPTokenizer.from_pretrained("openai/clip-vit-large-patch14")
    text_encoder = CLIPTextModel.from_pretrained("openai/clip-vit-large-patch14").to("cuda")
    # SDXL base model uses CLIP skip 2
    text_encoder.config.output_hidden_states = True

    tokenizer_2 = CLIPTokenizer.from_pretrained("laion/CLIP-ViT-bigG-14-laion2B-39B-b160k")
    text_encoder_2 = CLIPTextModel.from_pretrained("laion/CLIP-ViT-bigG-14-laion2B-39B-b160k").to("cuda")
    text_encoder_2.config.output_hidden_states = True

    # テキストエンコーディング
    prompt_embeds, negative_prompt_embeds, pooled_prompt_embeds, negative_pooled_prompt_embeds = pipe.encode_prompt(
        prompt,
        do_classifier_free_guidance=True, # classifier-free guidanceを使う場合
        negative_prompt="", # ネガティブプロンプトが必要な場合
        prompt_embeds=None,
        negative_prompt_embeds=None,
        pooled_prompt_embeds=None,
        negative_pooled_prompt_embeds=None,
        lora_scale=None,
        clip_skip=2, # 必要に応じて変更
        tokenizer=tokenizer,
        text_encoder=text_encoder,
        tokenizer_2=tokenizer_2,
        text_encoder_2=text_encoder_2,
    )

    # 追加の条件を準備 (タイムステップやオリジナルサイズなど)
    # これらの値はモデルや生成したい画像のサイズによって調整する必要があります。
    # ここでは一般的なSDXLのデフォルト値に近いものを例として設定します。
    original_size = (1024, 1024)
    target_size = (1024, 1024)
    crops_coords_top_left = (0, 0)
    aesthetic_score = 6.0 # 必要に応じて調整

    added_cond_kwargs = {
        "text_embeds": pooled_prompt_embeds,
        "time_ids": pipe._get_add_time_ids(
            original_size, crops_coords_top_left, target_size, aesthetic_score, torch.float16
        ).to("cuda")
    }

    # プロンプトで画像生成
    # prompt_embeds と negative_prompt_embeds を直接渡します。
    image = pipe(
        prompt_embeds=prompt_embeds,
        negative_prompt_embeds=negative_prompt_embeds,
        num_inference_steps=50, # ステップ数を指定
        guidance_scale=7.5, # guidance scaleを指定
        added_cond_kwargs=added_cond_kwargs
    ).images[0]

except Exception as e:
    print(f"Error occurred while preparing SDXL specific inputs: {e}")
    print("Attempting generation without added_cond_kwargs (may not work for SDXL models).")
    # SDXLではないモデルの場合やエラー時の代替処理
    # プロンプトで画像生成 (added_cond_kwargsなし)
    image = pipe(prompt).images[0]


image.save("output.png")

Fetching 17 files:   0%|          | 0/17 [00:00<?, ?it/s]

Loading pipeline components...:   0%|          | 0/5 [00:00<?, ?it/s]

You have disabled the safety checker for <class 'diffusers.pipelines.stable_diffusion.pipeline_stable_diffusion.StableDiffusionPipeline'> by passing `safety_checker=None`. Ensure that you abide to the conditions of the Stable Diffusion license and do not expose unfiltered results in services or applications open to the public. Both the diffusers team and Hugging Face strongly recommend to keep the safety filter enabled in all public facing circumstances, disabling it only for use-cases that involve analyzing network behavior or auditing its results. For more information, please have a look at https://github.com/huggingface/diffusers/pull/254 .


OutOfMemoryError: CUDA out of memory. Tried to allocate 26.00 MiB. GPU 0 has a total capacity of 14.74 GiB of which 26.12 MiB is free. Process 56153 has 14.71 GiB memory in use. Of the allocated memory 14.29 GiB is allocated by PyTorch, and 309.68 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)